# PCBI Demo Plugin
## Introduction
This is a Tutorial on how to write a Plugin for the free Demo version of PCB Investigator.
For further information please visit our website https://www.pcb-investigator.com/de and look into the PCB Investigator Documentation. The Documentation is either found within the download from below or at https://www.pcb-investigator.com/de/documentation.
To Test things out yourself, you can download the PCBI Demo SDK from
https://www.pcb-investigator.com/de/sdk-participate

This Plugin calculates distances between Tespoints and between Testpoints and Components.
The necessary Code already existed. I only put it to use in a GUI. You can find this and many more Scripts in our GitHub Script Collection https://github.com/sts-CAD-Software/PCB-Investigator-Scripts.
I started with the following Script: https://github.com/sts-CAD-Software/PCB-Investigator-Scripts/blob/master/DFM/Testpoint_CheckCellOutline2CellOutline.cs

## First Steps

Download the SDK from above Link. And open the PCBI Project in Visual Studio (tested with VS15 and VS17).
Compile and Run to look at it. Klick around and make yourself familiar with the Tool.
If you have problems compiling try following:

    - in the project properties set the target framework to a newer .NET version.
    - maybe the Newtonsoft.Json.dll is missing. Copy it into the binary folder(Debug)

For startes we will begin with something easy.
In the Project find PluginToolBarDemo.cs.

In [ ]:
public PluginToolBarDemo()
        {

            this.BackColor = Color.Red; //for the demo we want to highlight the tool bar
            this.Items.Add("Top View");
            this.Items[0].Click += new EventHandler(PluginToolBarDemoTopView_Click);
            position = ContainerPosition.Top;
            this.Items.Add("Bot View");
            this.Items[1].Click += new EventHandler(PluginToolBarDemoBotView_Click);
            this.Items.Add("Hallo");
            this.Items[2].Click += new EventHandler(HelloWorld_Click);
        }

Here i added the last two lines. This adds a button with "Hallo" on it and an eventhandler for clicking the button.

<img src = "files/imagesForNotebook/PCBI_Menu.png">

now add a function to the eventhandler

In [ ]:
private void HelloWorld_Click(object sender, EventArgs e)
        {
            MessageBox.Show("Hello World.", MessageBoxButtons.OK, MessageBoxIcon.Information)
        }

Compile and run. Now if you click on "Hallo" a message box appears to say hello.
You can insert any code you like.

In above Picture you can see the open Plugin tab. That's where we want our Plugins.
To see what the Script does, copy it into a new File in your Project. For example MyScript.cs like this.

In [ ]:
//Synchronous template
//-----------------------------------------------------------------------------------
// PCB-Investigator Automation Script
// Created on 2017-04-26
// Author EasyLogix
// www.pcb-investigator.com
// SDK online reference http://www.pcb-investigator.com/sites/default/files/documents/InterfaceDocumentation/Index.html
// SDK http://www.pcb-investigator.com/sdk-participate
// Updated: 2017-06-13
// Description: Check the distance between components and testpoints.
//-----------------------------------------------------------------------------------
// Favorite scripts can be placed in the ribbon menu. Therefore a unique GUID and a ButtonEnabled state is mandatory:
// GUID Testpoint_Check_636288129921985500
// ButtonEnabled=5   (Button enabled if: 1=Design is loaded, 2=Always, 4=Design contains components, 8=Loaded step is a panel, 16=Element is selected, 32=Component is selected)

using System;
using System.Collections.Generic;
using System.Text;
using PCBI.Plugin;
using PCBI.Plugin.Interfaces;
using System.Windows.Forms;
using System.Drawing;
using PCBI.Automation;
using System.IO;
using System.Drawing.Drawing2D;
using PCBI.MathUtils;

namespace PCBIScript
{
    public class MyScript
    {
        private const string tpReferencePrexix = "TP";  // Testpunkt Ref Prefix
        private double MinDistanceTP2CMP = PCBI.MathUtils.IMath.MM2Mils(5);  // Abstand zu Bauteil
        private double MinDistanceTP2TP = PCBI.MathUtils.IMath.MM2Mils(0.8);  // Abstand TP zu TP
        private int tpCountTotal = 0;
        public MyScript()
        {
        }

        public void Execute(IPCBIWindow parent)
        {
            if (parent == null || !parent.JobIsLoaded)
            {
                MessageBox.Show("No job loaded.");
                return;
            }

            IStep step = parent.GetCurrentStep();
            IMatrix matrix = parent.GetMatrix();

            if (matrix == null || step == null)
            {
                MessageBox.Show("No job loaded.");
                return;
            }

            PCB_Investigator.PCBIWindows.PCBIReportDialog reportDlg = new PCB_Investigator.PCBIWindows.PCBIReportDialog("TP Check Cell Outline to Cell Outline and TP to high Componen", "Result:");


            //Check Top Component Layer
            tpCountTotal = 0;
            string topComponentLayerName = matrix.GetTopComponentLayer();
            if (!string.IsNullOrWhiteSpace(topComponentLayerName))
            {
                ILayer dataLayer = step.GetLayer(topComponentLayerName);
                if (dataLayer != null && dataLayer is ICMPLayer)
                {
                    CheckTestpoints((ICMPLayer)dataLayer, ref reportDlg);
                }
            }

            //Check Bot Component Layer
            string botComponentLayerName = matrix.GetBotComponentLayer();
            if (!string.IsNullOrWhiteSpace(botComponentLayerName))
            {
                ILayer dataLayer = step.GetLayer(botComponentLayerName);
                if (dataLayer != null && dataLayer is ICMPLayer)
                {
                    CheckTestpoints((ICMPLayer)dataLayer, ref reportDlg);
                }
            }

            parent.UpdateView();
            reportDlg.ShowDlg(PCB_Investigator.PCBIWindows.PCBIReportDialog.WindowType.Modal);
        }

        private void CheckTestpoints(ICMPLayer layer, ref PCB_Investigator.PCBIWindows.PCBIReportDialog reportDlg)
        {
            if (layer == null)
            {
                MessageBox.Show("Layer is null referenced.");
                return;
            }

            reportDlg.AppendLog("Analyzing " + layer.GetLayerName() + "          -*-*-*-*-*-*-*-*-*-*-*-*-*---------------------------------");

            PointD fromPoint = PointD.Empty, toPoint = PointD.Empty;

            List<IObject> allComponents = layer.GetAllLayerObjects();
            double maxDistance = Math.Max(MinDistanceTP2TP, MinDistanceTP2CMP) * 1.01;

            int tpCountOK = 0;
            foreach (IObject element in allComponents)
            {
                if (element is ICMPObject)
                {
                    ICMPObject tpCmp = (ICMPObject)element;

                    if (tpCmp.Ref.StartsWith(tpReferencePrexix))  //cmp is a Testpoint...
                    {
                        tpCountTotal++;
                        bool hasWrongDistance = false;
                        RectangleD checkRect = tpCmp.GetBoundsD();
                        checkRect.Inflate(maxDistance, maxDistance);

                        IPolyClass tpPoly = tpCmp.GetPolygonOutline(false);

                        foreach (IObject nearElement in layer.GetAllObjectInRectangle(checkRect))
                        {
                            if (nearElement is ICMPObject)
                            {
                                ICMPObject nearCmp = (ICMPObject)nearElement;
                                if (tpCmp.Ref == nearCmp.Ref) continue;
                                if (!nearCmp.Ref.StartsWith(tpReferencePrexix) && nearCmp.CompHEIGHT < PCBI.MathUtils.IMath.MM2Mils(5)) continue; // Abfrage ob Bauteil kleiner 5 mm, dann ignorieren ausser Testpunkte

                                IPolyClass nearCmpPoly = nearCmp.GetPolygonOutline(false);

                                if (tpPoly != null && nearCmpPoly != null)
                                {
                                    double distance = tpPoly.DistanceTo(nearCmpPoly, ref fromPoint, ref toPoint);
                                    if (nearCmp.Ref.StartsWith(tpReferencePrexix))  //nearCmpPoly is also a Testpoint...
                                    {
                                        if (distance < MinDistanceTP2TP)
                                        {
                                            if (hasWrongDistance == false)
                                            {
                                                tpCmp.ObjectColor = Color.Orange;
                                                hasWrongDistance = true;
                                            }
                                            reportDlg.AppendLog(" -> " + tpCmp.Ref + " too close to " + nearCmp.Ref + " (" + PCBI.MathUtils.IMath.Mils2MM(distance).ToString("F3") + "mm)", PCB_Investigator.PCBIWindows.PCBIReportDialog.LogType.Warning);
                                        }

                                    }
                                    else
                                    {
                                        if (distance < MinDistanceTP2CMP)
                                        {
                                            if (hasWrongDistance)
                                            {
                                                tpCmp.ObjectColor = Color.Red;
                                                hasWrongDistance = true;
                                            }
                                            reportDlg.AppendLog(" -> " + tpCmp.Ref + " too close to " + nearCmp.Ref + " (" + PCBI.MathUtils.IMath.Mils2MM(distance).ToString("F3") + "mm)", PCB_Investigator.PCBIWindows.PCBIReportDialog.LogType.Warning);
                                        }
                                    }
                                }
                            }
                        }

                        if (!hasWrongDistance)
                        {
                            tpCountOK++;
                            tpCmp.ObjectColor = Color.Green;
                            reportDlg.AppendLog(" -> " + tpCmp.Ref + " is OK");
                        }
                    }
                }
            }

            reportDlg.AppendLog("=> " + tpCountOK + " of " + tpCountTotal + " Testpoints are OK **************************************************");
        }
    }
}

Now change your HelloWorld method to:

In [ ]:
private void HelloWorld_Click(object sender, EventArgs e)
        {
            PCBIScript.MyScript myScript = new PCBIScript.MyScript();
            myScript.Execute(parent);
        }

After compiling and running, load an example Project. You should get an output similar to this, except if your testpoints have a different prefix or you have no testpoints in your Project. Then the output will be significantly shorter.
Additionaly the Script changes the color of the checked testpoints. 

<img src = "files/imagesForNotebook/PCBI_ScriptOutput.png">

## New Plugin Button and GUI
First we want our Button to be in the PlugIn tab. For that you have to go into the PluginMenuItemDemo.cs

In [ ]:
void IPluginRibbonCommand.RegisterCommand()
        {
            regEntryPCBI_all = Parent.UIAction.RegisterID(new PCBI.Plugin.Interfaces.IRegisterItem() { Text = "Demo Item All", MetroStyleBackground = PCB_Investigator.PCBIStyleLibrary.PCBIColors.GetStandardColor(), MetroStyleIcon = PCB_Investigator.PCBIStyleLibrary.PCBIResources.AboutIcon(), ToolTip = "This is a example for PCB-Investigator.", RegisterType = RegisterItemType.BUTTON, GUID = "D13A03D3-36F0-4306-9F7B-352681DC515C", Category = "Other", EnableOn = Availability.JobOpen });
            regEntryPCBI_top = Parent.UIAction.RegisterID(new PCBI.Plugin.Interfaces.IRegisterItem() { Text = "Demo Item Top", MetroStyleBackground = PCB_Investigator.PCBIStyleLibrary.PCBIColors.GetStandardColor(), MetroStyleIcon = PCB_Investigator.PCBIStyleLibrary.PCBIResources.PCB_TopView_Icon(), ToolTip = "This is a example for PCB-Investigator.", RegisterType = RegisterItemType.BUTTON, GUID = "D13A03D3-36F0-4307-9F7B-352681DC515C", Category = "Other", EnableOn = Availability.JobOpen });
            regEntryPCBI_bot = Parent.UIAction.RegisterID(new PCBI.Plugin.Interfaces.IRegisterItem() { Text = "Demo Item Bot", MetroStyleBackground = PCB_Investigator.PCBIStyleLibrary.PCBIColors.GetStandardColor(), MetroStyleIcon = PCB_Investigator.PCBIStyleLibrary.PCBIResources.BottomViewIcon(), ToolTip = "This is a example for PCB-Investigator.", RegisterType = RegisterItemType.BUTTON, GUID = "D13A03D3-36F0-4308-9F7B-352681DC515C", Category = "Other", EnableOn = Availability.JobOpen });
            regEntryPCBI_net = Parent.UIAction.RegisterID(new PCBI.Plugin.Interfaces.IRegisterItem() { Text = "Demo Item Net", MetroStyleBackground = PCB_Investigator.PCBIStyleLibrary.PCBIColors.GetStandardColor(), MetroStyleIcon = PCB_Investigator.PCBIStyleLibrary.PCBIResources.NetIcon(), ToolTip = "This is a example for PCB-Investigator.", RegisterType = RegisterItemType.BUTTON, GUID = "D13A03D3-36F0-4309-9F7B-352681DC515C", Category = "Other", EnableOn = Availability.ObjectSelected });
            regEntryPCBI_TP_CO2CO = Parent.UIAction.RegisterID(new PCBI.Plugin.Interfaces.IRegisterItem() { Text = "Test Point analysis Cell outline to Cell outline", MetroStyleBackground = PCB_Investigator.PCBIStyleLibrary.PCBIColors.GetStandardColor(), MetroStyleIcon = new Icon("Icons/TP2CMP_16.ico"), ToolTip = "This checks distances between testpoints and components.", RegisterType = RegisterItemType.BUTTON, GUID = "D13A03D3-36F0-4311-9F7B-352681DC515C", Category = "Other", EnableOn = Availability.JobOpen });
            regEntryPCBI_embedded = Parent.UIAction.RegisterID(new PCBI.Plugin.Interfaces.IRegisterItem() { Text = "Demo Item Embedded", MetroStyleBackground = PCB_Investigator.PCBIStyleLibrary.PCBIColors.GetStandardColor(), MetroStyleIcon = PCB_Investigator.PCBIStyleLibrary.PCBIResources.PCBIEmbeddedIcon(), ToolTip = "This is a example for PCB-Investigator.", RegisterType = RegisterItemType.BUTTON, GUID = "D13A03D3-36F0-4310-9F7B-352681DC515C", Category = "Other", EnableOn = Availability.JobOpen });
        }

Add a new regEntry with a Name you like. I added the regEntryPCBI_TP_CO2CO. For that you just have to copy paste one of the above lines. and Change a few things:
1. Text = "Put in the name of your Button"
2. You may want your own Icon so change the MetroStyleIcon = ... accordingly
3. Change the Tooltip to what you want it to say
4. Change the GUID. this is Important else it will give you an error. There must not be two identical GUIDs
5. Decide when your Button/Plugin should be available. In this case Availability.JobOpen is good.

In [ ]:
void IPluginRibbonCommand.OnCommandExecute(int cmdID)
{
            if (cmdID == regEntryPCBI_all)
                ActivateAllSignalLayer();
            else if (cmdID == regEntryPCBI_top)
                ActivateTopCMPLayer();
            else if (cmdID == regEntryPCBI_bot)
                SelectAllObjectsOnBotCMPLayer();
            else if (cmdID == regEntryPCBI_net)
                NetSelection();
            else if (cmdID == regEntryPCBI_embedded)
                saveAsEmbedded();
            else if (cmdID == regEntryPCBI_TP_CO2CO)
                MyTool();
}

In [ ]:
        int regEntryPCBI_all = -1;
        int regEntryPCBI_top = -1;
        int regEntryPCBI_bot = -1;
        int regEntryPCBI_net = -1;
        int regEntryPCBI_TP_CO2CO = -1;
        int regEntryPCBI_embedded = -1; 

Add the else if statement with your cmdID and the function you want to run and the property with your regEntry.
Create a new function in the Events region:

In [ ]:
private void MyTool()
        {
        }

Now you should find a new Button in the PlugIn tab in the PCB Investigator.
But yet it has to do something. Add a new WinForm and design it like you want. For this example i will use my WinForm and the related code. Included in the SDK download package you will find the PCB Documentation. Use it for reference purposes.

<img src = "files/imagesForNotebook/PCBI_PluginGUI.png">

I used a modified version of the listview by inserting a normal listview and changing the definition in the ...Designer.cs.

In [ ]:
private PCB_Investigator.PCBIControls.ListViewFilter.ListViewFilter listView1;

Additionaly there are three textboxes for user input of the testpoint prefix and the desired minimum distances. Also there are three buttons to start the test, close the window and open the manual. Last but not least we have a few labels to show what is what.

## Implementing Functionality
With Form and Button ready we only need to implement the Plugin itself.
Lets start with showing the Form in PCB Investigator when the button is clicked.
Just change your MyTool() function in PluginMenuItemDemo.cs to the following.

In [ ]:
private void MyTool()
        {
            FormCheckCellOutline2CellOutline myform = new FormCheckCellOutline2CellOutline(parent);
            myform.ShowDialog();
        }

I used ShowDialog(), to increase usability. Now if you click your newly made Button the Form should show.
But let's start. Following namespaces have to be included.

In [ ]:
using System;
using System.Globalization;
using System.Collections.Generic;
using System.Drawing;
using System.Windows.Forms;
using PCBI.Automation;
using PCBI.MathUtils;

### Properties and Constructor of the Form

In [ ]:
namespace PluginDemo
{

    public partial class FormCheckCellOutline2CellOutline : Form
    {
        private IPCBIWindow parent;
        private string tpPrefix;
        private List<Color> lastColors = new List<Color>();
        private List<ICMPObject> lastObjects = new List<ICMPObject>();
        private ToolTip MyToolTip;
        private bool hasWrongDistance;
        private double MinDistanceTP2CMP;// = PCBI.MathUtils.IMath.MM2Mils(5);  // Abstand zu Bauteil
        private double MinDistanceTP2TP;// = PCBI.MathUtils.IMath.MM2Mils(0.8);  // Abstand TP zu TP
        private bool usedUnit;
        private int tpCountTotal = 0;
        private int idCount = 0;

        public FormCheckCellOutline2CellOutline(IPCBIWindow parent)
        {
            InitializeComponent();
            this.parent = parent;

            listView1.AddColumn("ID", 25, HorizontalAlignment.Left, PCB_Investigator.PCBIControls.ListViewFilter.LVFDataType.Number, false);
            listView1.AddColumn("Testpoint 1", 50, HorizontalAlignment.Left, PCB_Investigator.PCBIControls.ListViewFilter.LVFDataType.Auto, false);
            listView1.AddColumn("Testpoint/Object 2", 50, HorizontalAlignment.Left, PCB_Investigator.PCBIControls.ListViewFilter.LVFDataType.Auto, false);
            listView1.AddColumn("Board Side", 80, HorizontalAlignment.Left, PCB_Investigator.PCBIControls.ListViewFilter.LVFDataType.String, false);
            listView1.AddColumn("Warning", 60, HorizontalAlignment.Left, PCB_Investigator.PCBIControls.ListViewFilter.LVFDataType.String, false);
            listView1.AddColumn("Distance", -2, HorizontalAlignment.Left, PCB_Investigator.PCBIControls.ListViewFilter.LVFDataType.Number, false);

            parent.PCBIJobClosed += Parent_PCBIJobClosed;
        }

The parent property is needed so the Plugin has access to the PCB Investigators Features and the loaded Project. The rest is for later use and will be explained then. 
In the Constructor the listview is filled with Columns to use as output. Each element, which was seen before in the LogBox created by the raw Script, will have an ID, the names of the Tespoints/Objects tested, the side of the board on which they lie, an indicator if the test was ok or if there was a violation and the distance between the tested objects.
With the last line i added a new Event.

### Buttons and Events

In [ ]:
 private void Parent_PCBIJobClosed(object sender, EventArgs e)
        {
            this.Close();
        }

Definition of above declared Event, so if the PCB Investigator window is closed the plugin also shuts down.

In [ ]:
private void FormCheckCellOutline2CellOutline_Activated(object sender, EventArgs e)
        {
            if (parent.GetUnit())
            {
                Lblmm0.Text = "mm";
                Lblmm1.Text = "mm";
            }
            else
            {
                Lblmm0.Text = "mils";
                Lblmm1.Text = "mils";
                TxtBoxMinDstTP2CMP.Text = "80"; 
                TxtBoxMinDstTP2TP.Text = "100";
            }
        }

This triggers when the form loads.
It is used to change the unit from metric to imperial.

In [ ]:
private void Event_KeyUp(object sender, KeyEventArgs e)
        {
            if (e.KeyCode == Keys.Enter)
            {
                CmdStart_Click(sender, e);
            }
            if (e.KeyCode == Keys.Escape) 
            {
                CmdCancel_Click(sender, e);
            }
        }

This is only to add usability. The User can press Enter to start the Tool and Escape to exit it.

In [ ]:
private void CmdCancel_Click(object sender, EventArgs e)
        {
            this.Close();
        }

A click on Cancel closes the window.

In [ ]:
private void CmdHelp_Click(object sender, EventArgs e)
        {
            System.Diagnostics.Process.Start("Manual.pdf");
        }

A click on the help button opens the Manual.

In [ ]:
private void CmdStart_Click(object sender, EventArgs e)
        {
            if (string.IsNullOrWhiteSpace(TxtBoxPrefix.Text))
            {
                MessageBox.Show("Please enter the prefix of your Testpoints");
                return;
            }
            else
                tpPrefix = TxtBoxPrefix.Text;


            usedUnit = parent.GetUnit();
            //parse Min Distances
            try
            {
                MinDistanceTP2CMP = NumToEU(TxtBoxMinDstTP2CMP.Text, usedUnit);
                MinDistanceTP2TP = NumToEU(TxtBoxMinDstTP2TP.Text, usedUnit);
            }
            catch (Exception exeption1)
            {
                MessageBox.Show("Invalid distance.", exeption1.Message, MessageBoxButtons.OK, MessageBoxIcon.Error);
                return;
            }

            IStep step = parent.GetCurrentStep();
            IMatrix matrix = parent.GetMatrix();

            listView1.DeleteAllItems();
            listView1.RemoveFilter();

            //Check Top Component Layer
            tpCountTotal = 0;
            idCount = 0;
            CheckLayer(step, matrix.GetTopComponentLayer());
            //Check Bot Component Layer
            tpCountTotal = 0;
            CheckLayer(step, matrix.GetBotComponentLayer());
            

            lastColors.Clear();
            lastObjects.Clear();
            parent.UpdateView();
        }


The Start button does a few things more. The if statement checks if anything was entered as prefix. The try-catch block converts the input of the two textboxes containing the minimum distances into numbers. The NumToEU method is eplained below. After that the current step and matrix are fetched and the listview is cleared. Then top and bottom layer are checked by the script. 

In [ ]:
        public double NumToEU(string number, bool unit)
        {
            double newValue;
            NumberFormatInfo provider = new NumberFormatInfo();
            provider.NumberDecimalSeparator = ",";
            provider.NumberGroupSeparator = ".";

            if (number.IndexOf(",") < number.IndexOf("."))
            {
                char temp = '+';
                number = number.Replace(',', temp);
                number = number.Replace('.', ',');
                number = number.Replace(temp, '.');
            }

            try
            {
                newValue = Convert.ToDouble(number, provider);
            }
            catch (FormatException)
            {
                throw new FormatException("Unexpected characters.");
            }
            catch (OverflowException)
            {
                throw new OverflowException("Value out of range.");
            }

            if (unit)
            {
                return IMath.MM2Mils(newValue);
            }
            else
            {
                return newValue;
            }
        }

Here I used the method described in the documentation of Convert.ToDouble(number, provider). What happens essentially is, i define a convention how numbers are written and if the number i get uses another, i just switch ',' with '.' so it matches the other one. After that i convert the number via Convert.ToDouble and check if it's really a number. The conversion to mils is necessary, because in a later part of the program the distances are compared with a mils value.

In [ ]:
        private void CheckLayer(IStep step, string layerName)
        {
            if (!string.IsNullOrWhiteSpace(layerName))
            {
                ILayer dataLayer = step.GetLayer(layerName);
                if (dataLayer != null && dataLayer is ICMPLayer)
                {
                    CheckTestpoints((ICMPLayer)dataLayer);
                }
            }
        }

the CheckLayer method just checks a few exceptions and then calls CheckTestpoints.

### The actual Script
Now let's have a look at how the rest of the Script has changed. 

In [ ]:
        public void CheckTestpoints(ICMPLayer layer)
        {
            PointD fromPoint = PointD.Empty, toPoint = PointD.Empty;

            List<IObject> allComponents = layer.GetAllLayerObjects();
            double maxDistance = Math.Max(MinDistanceTP2TP, MinDistanceTP2CMP) * 1.01;

            int tpCountOK = 0;
            string layerName = layer.GetLayerName();
            foreach (IObject element in allComponents)
            {
                if (element is ICMPObject)
                {
                    ICMPObject tpCmp = (ICMPObject)element;

                    if (tpCmp.Ref.StartsWith(tpPrefix))  //cmp is a Testpoint...
                    {
                        tpCountTotal++;

                        hasWrongDistance = false;
                        RectangleD checkRect = tpCmp.GetBoundsD();
                        checkRect.Inflate(maxDistance, maxDistance);

                        IPolyClass tpPoly = tpCmp.GetPolygonOutline(false);

                        foreach (IObject nearElement in layer.GetAllObjectInRectangle(checkRect))
                        {

                            if (nearElement is ICMPObject)
                            {
                                ICMPObject nearCmp = (ICMPObject)nearElement;
                                if (tpCmp.Ref == nearCmp.Ref) continue;
                                // Check if component is smaller than 5mm
                                if (!nearCmp.Ref.StartsWith(tpPrefix) && nearCmp.CompHEIGHT < PCBI.MathUtils.IMath.MM2Mils(5)) 
                                    continue; 

                                IPolyClass nearCmpPoly = nearCmp.GetPolygonOutline(false);

                                if (tpPoly != null && nearCmpPoly != null)
                                {
                                    double distance = tpPoly.DistanceTo(nearCmpPoly, ref fromPoint, ref toPoint);
                                    CheckNearElements(tpCmp, nearCmp, distance, layerName);
                                }
                            }
                        }

                        if (!hasWrongDistance)
                        {
                            idCount++;
                            tpCountOK++;
                            tpCmp.ObjectColor = Color.Green;
                            AddListItem(idCount, tpCmp.Ref, "", layerName, "OK", "");
                        }
                        
                    }
                }
            }

            WriteLabels(tpCountOK, tpCountTotal, layerName);

        }

First two Points for a later calculation and a list containing all objects of the job are created. The max distance is going to be used shortly. After checking for each object in the list if it is an ICMPObject and if it is a testpoint 

In [ ]:
                        RectangleD checkRect = tpCmp.GetBoundsD();
                        checkRect.Inflate(maxDistance, maxDistance);

creates a rectangle with the bounds of the testpoint and inflates it by maxDistance on both axes. Using this Rectangle the list of objects to compare with is drastically reduced. Only Objects within the rectangle are checked, because anything further away is ok anyway and can be ignored.
Each of the Objects within the rectangle is again checked if it is an ICMPObject, if it is the same as the first one and if it is smaller than 5mm. If it passes all those checks the distance between the two objects is calculated. this calculation returns a value in mils. That's why the minDistances were saved in mils, too.

In [ ]:
private void CheckNearElements(ICMPObject tpCmp, ICMPObject nearCmp, double distance, string layerName)
{
            
            if (nearCmp.Ref.StartsWith(tpPrefix))  //nearCmpPoly is also a Testpoint...
            {
                if (distance < MinDistanceTP2TP)
                {
                    idCount++;
                    if (hasWrongDistance == false)
                    {
                        tpCmp.ObjectColor = Color.Orange;
                        hasWrongDistance = true;
                    }

                    if (usedUnit)
                    {
                        AddListItem(idCount, tpCmp.Ref, nearCmp.Ref, layerName, "X", 
                                    PCBI.MathUtils.IMath.Mils2MM(distance).ToString("F3"));
                    }
                    else
                    {
                        AddListItem(idCount, tpCmp.Ref, nearCmp.Ref, layerName, "X", 
                                    distance.ToString("F3"));
                    }

                }

            }
            else
            {
                if (distance < MinDistanceTP2CMP)
                {
                    idCount++;
                    if (hasWrongDistance == false)
                    {
                        hasWrongDistance = true;
                    }
                    tpCmp.ObjectColor = Color.Red;
                    if (usedUnit)
                    {
                        AddListItem(idCount, tpCmp.Ref, nearCmp.Ref, layerName, "X",
                                    PCBI.MathUtils.IMath.Mils2MM(distance).ToString("F3"));
                    }
                    else
                    {
                        AddListItem(idCount, tpCmp.Ref, nearCmp.Ref, layerName, "X", 
                                    distance.ToString("F3"));
                    }
                }
            }
        }

Depending on wether the object to compare is a testpoint or a component the correstponding minDistance value is used to check if it is far enough away. If it is not, it is marked with hasWrongdistance(if not already) so it doesn't get signed as ok, colored and an Element for the listview is created.

In [ ]:
        private void AddListItem(int ID, string TP1, string TP2, string layerName, string warning, string distance)
        {
            ListViewItem item = new ListViewItem(ID.ToString());
            item.SubItems.Add(TP1);
            item.SubItems.Add(TP2);
            item.SubItems.Add(layerName);
            item.SubItems.Add(warning);
            item.SubItems.Add(distance);
            listView1.Items.Add(item);
        }

Back to CheckTestpoints. If the the testpoint was checked against all elements and did not have any problems, it gets marked as ok and colored in green and added to the listview. 

In [ ]:
        private void WriteLabels(int tpCountOK, int tpCountTotal, string layerName)
        {
            IMatrix matrix = parent.GetMatrix();
            if (layerName.ToLower() == matrix.GetTopComponentLayer().ToLower())
            {
                LblOkTop.Text = ("Top: " + tpCountOK + " of " + tpCountTotal + " Testpoints are OK");
            }
            else
            {
                LblOkBot.Text = ("Bot: " + tpCountOK + " of " + tpCountTotal + " Testpoints are OK");
            }
        }

When all the checks are done, the two labels beneath the listview get their text reassigned.

### Adding Functionality
Up until now the tool can do a few things more than the script. Filtering is provided by the custom listview class, it has a manual attached and it looks nicer. But one thing is missing. It would be nice, if it was possible to select items from the listview so that they are shown in the mainwindow.

In [ ]:
        private void ValueSelected(object sender, EventArgs e)
        {
            //reset colors if previously changed by this function
            for (int i = 0; i < lastObjects.Count; i++)
            {
                if (lastObjects[i] != null)
                {
                    lastObjects[i].ObjectColor = lastColors[i];
                }
            }
            lastColors.Clear();
            lastObjects.Clear();
            parent.UpdateView();

            IStep step = parent.GetCurrentStep();
            IMatrix matrix = parent.GetMatrix();
            if (matrix == null || step == null)
            {
                MessageBox.Show("No job loaded.");
                return;
            }


            //Get Names of Points/Components
            string[] TPNames = { listView1.SelectedItems[0].SubItems[1].Text, listView1.SelectedItems[0].SubItems[2].Text };
            string layerName = listView1.SelectedItems[0].SubItems[3].Text;

            //Check layer
            if (!string.IsNullOrWhiteSpace(layerName))
            {
                ILayer dataLayer = step.GetLayer(layerName);
                if (dataLayer != null && dataLayer is ICMPLayer)
                {
                    foreach (string name in TPNames)
                    {
                        lastObjects.Add(GetObject((ICMPLayer)dataLayer, name));
                    }
                }
            }

            EnableLayer(step, matrix, layerName);

            //go to the objects
            ZoomObjects(lastObjects);

            //the objects shall be blue!
            foreach(ICMPObject TP in lastObjects)
            {
                if (TP != null)
                {
                    lastColors.Add(TP.ObjectColor);
                    TP.ObjectColor = Color.Blue;
                }
            }

            parent.UpdateView();
        }

This method triggers if an item in the listview is selected by doubleclick. First it resets colors if changed previously. Second is a little safety check. Then the names of the Testpoints are extracted from the listview.
For each of these names the corresponding Object is fetched by GetObject.

In [ ]:
        private ICMPObject GetObject(ICMPLayer layer, string name)
        {
            List<IObject> allComponents = layer.GetAllLayerObjects();
            foreach (IObject element in allComponents)
            {
                if (element is ICMPObject)
                {
                    ICMPObject tpCmp = (ICMPObject)element;
                    if (tpCmp.Ref == name)
                    {
                        return tpCmp;
                    }
                }
            }
            return null;
        }

This method iterates through all objects on a given layer and returns the one with the same name as given. If it can't find the Object, it returns null.

After getting the two objects the layer is made visible by EnableLayer.

In [ ]:
        private void EnableLayer(IStep step, IMatrix matrix, string layerName)
        {
            if (matrix != null && step != null)
            {
                string topComponentLayerName = matrix.GetTopComponentLayer();
                string botComponentLayerName = matrix.GetBotComponentLayer();

                //enable layer on which the object lies, disable the other one
                List<ILayer> visibleLayers = step.GetVisibleLayerList();
                if (layerName.ToLower() == topComponentLayerName.ToLower())
                {
                    try
                    {
                        ILayer botLayer = visibleLayers.Find(layer => layer.GetLayerName().ToLower() == botComponentLayerName.ToLower());
                        botLayer.DisableLayer();
                    }
                    catch { }
                    ILayer topLayer = step.GetLayer(topComponentLayerName);
                    topLayer.EnableLayer(true);
                }
                else
                {
                    try
                    {
                        ILayer topLayer = visibleLayers.Find(layer => layer.GetLayerName().ToLower() == topComponentLayerName.ToLower());
                        topLayer.DisableLayer();
                    }
                    catch { }
                    ILayer botLayer = step.GetLayer(botComponentLayerName);
                    botLayer.EnableLayer(true);
                }

            }
        }

EnableLayer compares the bottom layer name and the top layer name with the given layer name and enables the given one (or none, if any other layer is given) and disables the other one. 

When the right layer is enabled the view gets zoomed in to the two objects by ZoomObjects

In [ ]:
        private void ZoomObjects(List<ICMPObject> components)
        {
            RectangleD view = RectangleD.Empty;
            foreach (ICMPObject component in components)
            {
                if (component != null)
                {
                    if (view.IsEmpty)
                    {
                        view = component.GetBoundsD();
                    }
                    else
                    {
                        view = RectangleD.Union(view, component.GetBoundsD());
                    }
                }

            }
            view.Inflate(30, 40);
            parent.ZoomRect(view);
        }

ZoomObjects creates a minimal rectangle around all given objects. Then it inflates the rectangle a bit and sets the main window to it.

With zooming in done, the selected object(s) get a new color(blue) and the main window view gets updated.

This is the whole plugin. I hope this Documentation/Tutorial helped you understand how to write a plugin for PCB Investigator.